In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
!git clone https://github.com/raghav-thiruv/Omdena-Quantum-Self-Driving.git

Cloning into 'Omdena-Quantum-Self-Driving'...
remote: Enumerating objects: 45761, done.
remote: Counting objects: 100% (11671/11671), done.
remote: Compressing objects: 100% (11653/11653), done.
remote: Total 45761 (delta 32), reused 11645 (delta 17), pack-reused 34090
Receiving objects: 100% (45761/45761), 2.16 GiB | 21.21 MiB/s, done.
Resolving deltas: 100% (61/61), done.
Updating files: 100% (45593/45593), done.


In [4]:
!ls

Omdena-Quantum-Self-Driving  __notebook_source__.ipynb


In [5]:
!mv "Omdena-Quantum-Self-Driving/Images/driving_dataset1/data.txt" "Omdena-Quantum-Self-Driving/Images/"

In [6]:
from struct import unpack
from tqdm import tqdm
import os

img_dir = 'Omdena-Quantum-Self-Driving/Images/driving_dataset1/'
data_file = "Omdena-Quantum-Self-Driving/Images/data.txt"

In [7]:
import tensorflow as tf
from tensorflow.keras.applications import VGG19
from tensorflow.keras.layers import Input, Flatten, Dense, Concatenate, Dropout
from tensorflow.keras.models import Model

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [8]:
def build_model():
    base_model = VGG19(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

    for layer in base_model.layers:
        layer.trainable = True

    flatten_output = Flatten()(base_model.output)

    angle_input = Input(shape=(1,))
    fused_output = Concatenate()([flatten_output, angle_input])

    fc1 = Dense(512, activation='relu')(fused_output)
    dropout1 = Dropout(0.3)(fc1)
    fc2 = Dense(256, activation='relu')(fc1)
    output = Dense(1)(fc2)

    model = Model(inputs=[base_model.input, angle_input], outputs=output)

    return model


# Load and preprocess a single image
def load_image(image_path):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, (224, 224))
    image = tf.keras.applications.vgg19.preprocess_input(image)
    return image

def create_dataset(image_files, angles):
    image_paths = [os.path.join(img_dir, file_name) for file_name in image_files]
    image_dataset = tf.data.Dataset.from_tensor_slices(image_paths)
    angle_dataset = tf.data.Dataset.from_tensor_slices(angles)
    image_dataset = image_dataset.map(load_image, num_parallel_calls=tf.data.AUTOTUNE)
    dataset = tf.data.Dataset.zip((image_dataset, angle_dataset))
    return dataset

# Get the list of image files
image_files = []

# Load the steering angles
angles_dict = {}
with open(data_file, 'r') as file:
    for line in file:
        img_file, angle = line.strip().split()
        image_files.append(img_file)
        angles_dict[img_file] = float(angle)

# Get the angles for the image files
angles = [angles_dict[file_name] for file_name in image_files]

# Split the image files into training and validation sets
validation_split = 0.2
num_validation_samples = int(validation_split * len(image_files))
train_image_files = image_files[num_validation_samples:]
train_image_angles = angles[num_validation_samples:]
val_image_files = image_files[:num_validation_samples]
val_image_angles = angles[:num_validation_samples]

# Create the training and validation datasets with both inputs
train_dataset = create_dataset(train_image_files, train_image_angles)
val_dataset = create_dataset(val_image_files, val_image_angles)

# Shuffle and batch the datasets
batch_size = 32
train_dataset = train_dataset.shuffle(buffer_size=1000).batch(batch_size)
val_dataset = val_dataset.batch(batch_size)

# Define the input shape for the angle dataset
angle_input_shape = (1,)

# Create the VGG-16 model
model = build_model()

# Define the mean squared error as the loss function
def mse_loss(y_true, y_pred):
    return tf.keras.losses.mean_squared_error(y_true, y_pred)

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), loss=mse_loss)

# Modify the fit call to pass both inputs
train_dataset = train_dataset.map(lambda img, angle: ((img, angle), angle))
val_dataset = val_dataset.map(lambda img, angle: ((img, angle), angle))

80134624/80134624 [==============================] - 1s 0us/step


In [9]:
# Train the model
epochs = 50
model.fit(train_dataset, validation_data=val_dataset, epochs=epochs)

Epoch 1/50
1136/1136 [==============================] - 343s 282ms/step - loss: 71.4111 - val_loss: 359.8236
Epoch 2/50
1136/1136 [==============================] - 323s 281ms/step - loss: 1.0737 - val_loss: 291.2111
Epoch 3/50
1136/1136 [==============================] - 313s 273ms/step - loss: 0.0380 - val_loss: 290.9698
Epoch 4/50
1136/1136 [==============================] - 323s 281ms/step - loss: 0.0313 - val_loss: 310.1879
Epoch 5/50
1136/1136 [==============================] - 313s 273ms/step - loss: 0.8157 - val_loss: 182.8761
Epoch 6/50
1136/1136 [==============================] - 314s 273ms/step - loss: 0.1461 - val_loss: 157.3448
Epoch 7/50
1136/1136 [==============================] - 323s 281ms/step - loss: 0.0081 - val_loss: 161.6421
Epoch 8/50
1136/1136 [==============================] - 323s 281ms/step - loss: 1.0842 - val_loss: 158.7609
Epoch 9/50
1136/1136 [==============================] - 313s 272ms/step - loss: 0.8085 - val_loss: 61.7568
Epoch 10/50
1136/1136 [=====

In [ ]:
import random
from tensorflow.keras.preprocessing import image as keras_image
import numpy as np

def load_image(img_dir, target_size=(224, 224)):
    img = keras_image.load_img(image_path, target_size=target_size)
    img_tensor = keras_image.img_to_array(img)
    img_tensor = np.expand_dims(img_tensor, axis=0)
    img_tensor = tf.keras.applications.resnet.preprocess_input(img_tensor)
    return img_tensor

# Randomly select an image from validation set
random_image_path = random.choice(val_image_files['filename'].tolist())

# Load the image
test_image = load_image(random_image_path)

# Use the model to predict the steering angle for the test image
predicted_angle = model.predict(test_image)

# Print out the predicted steering angle
print("Predicted steering angle: ", predicted_angle[0][0])

# If you want to compare this prediction to the actual angle, you could find that as follows:
actual_angle = val_image_files[val_image_files['filename'] == random_image_path]['steering_angle'].values[0]
print("Actual steering angle: ", actual_angle)
